In [43]:
import wikipediaapi as wiki
from tqdm import tqdm_notebook
import pymorphy2
import numpy as np
import nltk
import string

In [4]:
wiki_wiki = wiki.Wikipedia('ru')
page_py_1 = wiki_wiki.page('Категория:Кухни народов мира')
page_py_2 = wiki_wiki.page('Категория:Кухни по странам')

In [5]:
def get_categorymembers(categorymembers, level=0, max_level=0):
        return categorymembers.values()

cuisines_1 = get_categorymembers(page_py_1.categorymembers)
cuisines_2 = get_categorymembers(page_py_2.categorymembers)

cuisines_list = []
for page in cuisines_1:
    if 'Категория:' in page.title:
        cuisines_list.append(page.title)

for page in cuisines_2:
    if 'Категория:' in page.title:
        cuisines_list.append(page.title)

In [6]:
cuisines_list = list(set(cuisines_list))

In [7]:
cuisines_list[:5]

['Категория:Ирландская кухня',
 'Категория:Индейская кухня',
 'Категория:Индийская кухня',
 'Категория:Латышская кухня',
 'Категория:Норвежская кухня']

In [ ]:
import re

dishes_by_countries = {}
f = open('cuisines.txt', 'w')

for cuisine in cuisines_list:
    print(cuisine)
    dishes = get_categorymembers(wiki_wiki.page(cuisine).categorymembers)
    for dish in dishes:
        summary = re.sub('^.*? — ', '', dish.summary)
        punct = '[!"#$%&()*+,./:;<=>?@[\]^_`{|}~„“«»†*/\—–‘’]'
        nums = '[0-9]'
        summary = re.sub('\n', ' ', summary)
        summary = re.sub(punct, '', summary)
        summary = re.sub(nums, '', summary)
        summary = re.sub(r'(?![А-яё ]).', '', summary)
        try:
            f.write(summary + " ")
        except Exception:
            pass  
    f.write('\n\n\n')

In [69]:
with open('cuisines.txt', 'r', encoding='cp1251') as f:
    text = f.read()
    
texts = text.split('\n\n\n')
texts[0]

'сладкий дрожжевой хлеб с виноградом белым кишмишем и изюмом Обычно подаётся как тосты жареные ломтики со сливочным маслом к чаю Тесто для бармбрэка делается слаще чем для простого хлеба но не такое сладкое как для кексов Виноград и изюм придают этому хлебу особый аромат традиционное ирландское блюдо которое готовится из беконной части свиной туши специй гвоздика душистый перец и капусты иногда с добавлением картофеля моркови лука репы иногда для приготовления этого блюда используется копчёный бекон Блюдо обычно подаётся с белым соусом который обычно готовят из муки сливочного масла молока и петрушки Популярность этого блюда в прошлом объясняется тем что ингредиенты были доступны для большинства населения это блюдо и сейчас очень популярно в Ирландии это мясное блюдо пудинг популярное в Ирландии Шотландии Нортумберленде Новой Шотландии и Ньюфаундленде Белый пудинг во многом похож на чёрный пудинг но не содержит крови Согласно современным рецептам белый пудинг состоит из сала или жира о

In [70]:
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yaroslav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
word_tokenizer = nltk.WordPunctTokenizer()

In [72]:
dates = [str(x) for x in np.arange(1900, 2022)]
def process_data(data):
    texts = []
    targets = []
    
    # поочередно проходим по всем новостям в списке
    for item in data:   
        tokens = word_tokenizer.tokenize(item)
        tokens = [word for word in tokens if (word not in string.punctuation and word not in stop_words and word not in dates)]
        texts.append(tokens) # добавляем в предобработанный список
    
    return texts

In [73]:
# запускаем нашу предобработку
texts = process_data(texts)

In [74]:
morph = pymorphy2.MorphAnalyzer()
tokenized_texts = []

In [75]:
for i in tqdm_notebook(range(len(texts))):
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]]
    tokenized_texts.append(text_lemmatized)

C:\Users\yaroslav\AppData\Local\Temp\ipykernel_1136\135266771.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(texts))):


  0%|          | 0/150 [00:00<?, ?it/s]

In [76]:
tokenized_texts[0][:10]

['сладкий',
 'дрожжев',
 'хлеб',
 'виноград',
 'белый',
 'кишмиш',
 'изюм',
 'обычно',
 'подаваться',
 'тост']

In [89]:
with open('rus_stopwords.txt', 'r', encoding='utf-8') as f:
    sw = f.read().split('\n')

In [90]:
for ind, text in enumerate(tokenized_texts):
    text = [w for w in text if w not in sw]
    tokenized_texts[ind] = text

In [91]:
with open(r'text_lemmatized.txt', 'w') as fp:
    for item in tokenized_texts:
        fp.write("%s\n\n\n" % item)

In [92]:
from gensim.models import *
from gensim import corpora
from gensim import similarities

In [93]:
print('Making dictionary...')
dictionary = corpora.Dictionary(tokenized_texts)
print('Original: {}'.format(dictionary))
dictionary.filter_extremes(no_below = 5, no_above = 0.9, keep_n=None)
dictionary.save('polkrug.dict')
print('Filtered: {}'.format(dictionary))

print('Vectorizing corpus...')
corpus = [dictionary.doc2bow(text) for text in tokenized_texts]
corpora.MmCorpus.serialize('polkrug.model', corpus) 

Making dictionary...
Original: Dictionary(15890 unique tokens: ['англ', 'английский', 'англия', 'англоязычный', 'апельсиновый']...)
Filtered: Dictionary(4194 unique tokens: ['англ', 'английский', 'англия', 'англоязычный', 'апельсиновый']...)
Vectorizing corpus...


In [94]:
len(tokenized_texts), len(corpus)

(150, 150)